In [1]:
import ijson
import json


In [2]:
def ptoutlist_2invol(filename: str):
    lenofdata=15
    filedata=open(filename, 'r')
    results=[]
    datas = ijson.items(filedata, 'item')
    for data in datas:
        results.append(data)
    filedata.close()
    for key in data.keys():
        print(lenofdata)
        if "," in key:
            for y in key.split(","):
                if len(y)>lenofdata:
                    lenofdata=len(y)+1
        else:
            if len(key)>lenofdata:
                lenofdata=len(y)+1
    for key in data.keys():
        # H8 NID和REF都有
        if "," in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=(" "+key).split(",")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        else:
            print("这些键值没有输出：",key)

In [3]:


# 对输出结果记录文件进行整理
# 具有相当的特殊性，不适用于普遍情况
def ptoutlist_new(h7orh8: str,required3: bool, NIDorREF: str, srli=None, refnum=None):
    lenofdata=15
    
    datapath="../result/"
    if NIDorREF=="NID" or NIDorREF=="nid":
        if srli==None:
            print("Both sr ideal and linear ideal ? The fourth arg needed")
            return
        if h7orh8=="h7":
            if srli==None:
                print("Both sr ideal and linear ideal ? The fourth arg needed")
                return
            datapath+="NIDresultsout_h2-7_required3"+str(required3)+"_srli"+str(srli)+".txt"
        elif h7orh8=="h8":
            datapath+="NIDresultsout_h8-12_required3"+str(required3)+"_srli"+str(srli)+".txt"
        else:
            print("arg 1 wrong, it should be either \"h7\" or \"h8\"")
            return
    elif NIDorREF=="REF" or NIDorREF=="ref":
        if h7orh8=="h7":
            if refnum==None:
                print("for h11<8, refnum is needed, it should be 1,2 or 3")
                return
            datapath+="REFresultsout_h2-7_required3"+str(required3)+"_ref"+str(refnum)+".txt"
        elif h7orh8=="h8":
            if refnum!=3:
                print("for h11>7, we only compute part of the 3 divisors reflection, so here we only count those result")
            datapath+="REFresultsout_h8-12_required3"+str(required3)+"_ref"+str(refnum)+".txt"
        else:
            print("arg 1 wrong, it should be either \"h7\" or \"h8\"")
            return
    else:
        print("arg 3 wrong, it should be either \"NID\" or \"REF\"")
        return
    
    filedata=open(datapath, 'r')
    results=json.load(filedata)

    lenall = len(results)


    for key in results[0].keys():
        listouts=[]
        # H8 NID和REF都有
        if "," in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=(" "+key).split(",")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        # 都有
        elif "N_T_F" in key:#["h11, polynumber, trinumber", "no_oplane, numo3, numo7, numo5, numo3o7, involnumber"]:
            print("***************************"*2, key)
            smallkeys=("N_T_F").split("_")
            for i in range(len(smallkeys)):
                strout=smallkeys[i]
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for result in results:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(result[key][i]))<lenofdata:
                        stroutpatch+=" "
                    strout+="&  "+str(result[key][i])+stroutpatch
                print(strout,"&", str(sum([int(x) for x in strout.split("&")[1:]])),"   replaceitto")
        # 都有
        elif key=="invol_divisor_type":
            # 输出结果为只需包含下列除子类型的involution
            # 0： 未讨论的其他类型除子
            # 1:  del Pezzo 1-8
            # 2:  willson
            # 3:  K3  
            # 4:  SD1
            # 5:  SD2
            # 6:  non-shrinkable rigid divisors
            # 7:  exact willson
            for _ in [1, 2, 3, 4, 5, 6, 7]:
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in [1, 2, 3, 4, 5, 6, 7]:
                    key_satisfy=[]
                    y_need=[i]
                    for keyy in invol_divisor_type.keys():
                        # 有一个对于种类的divisor就行
                        for x in keyy.split(","):
                            for y in y_need:
                                if str(y) in x:
                                    break
                            if str(y) in x:
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i-1].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str([1, 2, 3, 4, 5, 6, 7][x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")
                
            listouts=[]
            # 同样要求只包含
            # dP_n + K3 
            # dP_n + willson
            # willson + K3
            # dP_n + willson + K3
            # dP_n + exact willson
            # K3 + exact willson
            # dP_n + exact willson + K3
            for _ in [[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]]:
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in range(len([[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]])):
                    key_satisfy=[]
                    y_need=[[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]][i]
                    y_need_same=eval(str(y_need).replace("1", "6"))
                    for keyy in invol_divisor_type.keys():
                        for y in y_need:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # 注意前面是in，而这里是not in，说明所需的除子并不都在keyy里
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                            continue
                        for y in y_need_same:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # x有y，break或者，x到最后一个，break；如果y 没在x 里，说明这个里面就没有y
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str([[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]][x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")
            # 对non-shrinkable的dp divisor
            listouts=[]
            divisorneed=[]
            for x in [[1, 3], [1, 2], [2, 3], [1, 3, 2], [1, 7], [3, 7], [1, 3, 7]]:
                divisorneed.append([y+(0**(y!=1))*5 for y in x])
                listouts.append([])
            for x in range(lenall):
                invol_divisor_type=results[x][key]
                for i in range(len(divisorneed)):
                    key_satisfy=[]
                    y_need=divisorneed[i]
                    y_need_same=eval(str(y_need).replace("1", "6"))
                    for keyy in invol_divisor_type.keys():
                        for y in y_need:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # 注意前面是in，而这里是not in
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                            continue
                        for y in y_need_same:
                            for x in keyy.split(","):
                                if str(y) in x:
                                    break
                            if str(y) not in x:
                                # x有y，break或者，x到最后一个，break；如果y 没在x 里，说明这个里面就没有y
                                break
                        if str(y) in x:
                            # print(key)
                            key_satisfy.append(keyy)
                    number_combine=0
                    for x in key_satisfy:
                        number_combine+=invol_divisor_type[x]
                    listouts[i].append(number_combine)
            print("***************************"*2, key)
            for x in range(len(listouts)):
                listout = listouts[x]
                strout=str(divisorneed[x])+key+"& "
                stroutpatch=""
                while len(strout+stroutpatch)<lenofdata:
                    stroutpatch+=" "
                strout+=stroutpatch
                for listou in listout:
                    stroutpatch=""
                    while len("&  "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    strout+=str(listou)+stroutpatch+"&  "
                strout += str(sum(listout))
                print(strout, "replaceitto")



        # NID
        elif key=="hodge_split":
            print("***************************"*2, key)
            listouts=[]
            for _ in range(12):
                # 12个空集对应，hodge-的各个情况
                listouts.append([])
            for result in results:
                # print(result["h11, polynumber, trinumber"][0])
                for listout in listouts:
                    # 每来一个result（不同的h11），就默认append一个0
                    listout.append(0)
                for keyhere in result[key]:
                    # 如果这个result有某个对应的hodge-存在，就将0覆盖
                    listouts[eval(keyhere)[1]][-1]=result["hodge_split"][keyhere]
            for i in range(12):
                # 对每一个hodge-
                outstr ="hodge_-"+ str(i)+" & "
                
                for listou in listouts[i]:
                    stroutpatch=""
                    while len(" &   "+stroutpatch+str(listou))<lenofdata:
                        stroutpatch+=" "
                    outstr += (str(listou)+stroutpatch+ "   & ")
                outstr += str(sum(listouts[i]))
                print(outstr)
        else:
            print(key, "这个键没有列出表格")


In [26]:
ptoutlist_new("h8", True, "ref", refnum=3)

****************************************************** h11, polynumber, triangnumber
 h11           &  8           &  9           &  10          &  11          &  12           & 50    replaceitto
 polynumber    &  1486        &  1581        &  1554        &  1621        &  1692         & 7934    replaceitto
 triangnumber  &  6847        &  9833        &  12185       &  15638       &  14980        & 59483    replaceitto
****************************************************** numo3_ref, numo7_ref, numo3o7_ref, numo5_ref, numo9_ref, numo5o9_ref, involnumber
 numo3_ref     &  1091        &  1184        &  1059        &  873         &  683          & 4890    replaceitto
 numo7_ref     &  4885        &  5278        &  5819        &  4594        &  4456         & 25032    replaceitto
 numo3o7_ref   &  56031       &  76473       &  94375       &  119044      &  111699       & 457622    replaceitto
 numo5_ref     &  0           &  0           &  0           &  0           &  0            & 0    

In [ ]:
# 输出结果为只需包含下列除子类型的involution
# 0： 未讨论的其他类型除子
# 1:  del Pezzo 1-8
# 2:  willson
# 3:  K3  
# 4:  SD1
# 5:  SD2
# 6:  non-shrinkable rigid divisors
# 7:  exact willson

In [17]:
ptoutlist_new("h7", False, "NID", srli=True)

****************************************************** h11, polynumber, trinumber
 h11           &  8           &  9           &  10          &  11          &  12           & 50    replaceitto
 polynumber    &  38          &  40          &  34          &  22          &  24           & 158    replaceitto
 trinumber     &  80          &  112         &  79          &  85          &  79           & 435    replaceitto
****************************************************** hodge_split
hodge_-0 & 0            & 0            & 0            & 0            & 0            & 0
hodge_-1 & 48           & 76           & 53           & 49           & 32           & 258
hodge_-2 & 22           & 26           & 12           & 18           & 12           & 90
hodge_-3 & 14           & 7            & 10           & 6            & 37           & 74
hodge_-4 & 2            & 13           & 4            & 13           & 4            & 36
hodge_-5 & 0            & 0            & 3            & 0            & 